<img src="https://geodecode.com.br/wp-content/uploads/2021/12/brasildatacube.png" align="left" width="74"/>
<img src="https://brazil-data-cube.github.io/_images/logo-odc-bdc.png" align="right" width="74" />

<h1 style="color:#336699; text-align: center" >Annual Metting Use Case</h1>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    <sup>
        <a href="https://orcid.org/0000-0002-9916-1167">
            <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i>
        </a>
    </sup>
    <br/>
    Brazil Data Cube (BDC), National Institute for Space Research (INPE)
    <br/>
    <br/>
    <br/>
    <div style="width: 98%;">
        <div style="text-align: justify; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 10px;">
            <b>Explantion here....:</b>
            <div>
                Write write write write writewrite writewrite write. <a href="http://www.brazildatacube.org/" target="_blank">Site site site site site</a>, <i> Outras infos</i> .......
            </div>
        </div>
    </div>
</div>

In [1]:
library(sf)
library(rstac)
library(dplyr)

Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 8.2.1; sf_use_s2() is TRUE


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Set constants
<hr style="border:1px solid #0077b9;">

In [10]:
HARMONIZE_STAC_URL <- 'https://brazildatacube.dpi.inpe.br/harmonize/dev/stac/v1/'

START_DATE <- "2020-01-01"
END_DATE <- "2020-12-31"

# List Harmonize STAC collections (using RSTAC)
<hr style="border:1px solid #0077b9;">

In [3]:
all_collections <- stac(HARMONIZE_STAC_URL) %>%
  collections() %>%
  get_request()

for (collection in all_collections$collections){
  print(collection$id)
}

[1] "dengue_mortality_rate_mun_year_north-1"
[1] "total_prec_max_no_mun_epiweek-1"
[1] "Mavic3M_FlightHeight120m_Mosaic-1"
[1] "max_temper_max_no_mun_epiweek-1"
[1] "max_temper_mean_ne_mun_month-1"
[1] "Phantom3Advanced_FlightHeight120m_Mosaic-1"
[1] "Phantom3Advanced_FlightHeight120m-1"
[1] "chagas_disease_cases_mun_week_north-1"
[1] "max_temper_max_no_mun_epiweek_ras-1"
[1] "max_temper_mean_no_mun_month-1"
[1] "max_temper_max_no_mun_month-1"
[1] "chagas_disease_cases_mun_month_north-1"
[1] "max_temper_max_ne_mun_epiweek-1"
[1] "dengue_cases_mun_year_north-1"
[1] "chikungunya_cases_mun_year_north-1"
[1] "zika_cases_mun_month_north-1"
[1] "dengue_cases_mun_month_north-1"
[1] "chikungunya_cases_mun_month_north-1"
[1] "dengue_cases_mun_week_north-1"
[1] "max_temper_mean_no_mun_epiweek-1"
[1] "dengue_mortality_rate_mun_month_north-1"
[1] "zika_cases_mun_year_north-1"
[1] "chagas_disease_cases_mun_year_north-1"
[1] "zika_cases_mun_week_north-1"
[1] "total_prec_max_no_mun_month-1"
[1] "deng

# Obtain monthly climate data for each municipality
<hr style="border:1px solid #0077b9;">

### Set collection id (temperature, precipitation, dengue, zika, etc)

In [11]:
COLLECTION_ID <- "max_temper_mean_no_mun_month-1"
# COLLECTION_ID <- "total_prec_max_no_mun_month-1"
# COLLECTION_ID <- "total_prec_max_no_mun_epiweek-1"
# COLLECTION_ID <- "max_temper_mean_no_mun_epiweek-1"
# COLLECTION_ID <- "dengue_mortality_rate_mun_month_north-1"
# COLLECTION_ID <- "dengue_cases_mun_week_north-1"
# COLLECTION_ID <- "dengue_cases_mun_month_north-1"

### Search climate data on Harmonize STAC (Using RSTAC)

In [12]:
stac_search <- stac(HARMONIZE_STAC_URL) %>%
  stac_search(
    collections = COLLECTION_ID,
    datetime = paste(START_DATE, END_DATE, sep = "/"),
    limit = 200
  ) %>%
  get_request()

stac_search$context

$matched
[1] 12

$returned
[1] 12

### For each feature, retrieve geojson as dataframe

In [13]:
feature_to_geojson_csv <- function(feature) {
  datetime <- feature$properties$datetime
  geojson_href <- feature$assets$geojson$href
  
  as.data.frame(st_read(geojson_href)) %>%
    mutate(date = format(as.Date(datetime), "%Y-%m-%d"))
}

dfs <- lapply(X = stac_search$features, FUN = feature_to_geojson_csv)
dfs[1:2]

Reading layer `max_temper_mean_NO_mun_month_20201201' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/climate_indicators/temperature_era5land/max_temper_mean_no_mun_month/shapefiles/2020-12-01/max_temper_mean_no_mun_month_20201201.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `max_temper_mean_NO_mun_month_20201101' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/climate_indicators/temperature_era5land/max_temper_mean_no_mun_month/shapefiles/2020-11-01/max_temper_mean_no_mun_month_20201101.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `max_t

CD_MUN,NM_MUN,SIGLA,AREA_KM2,cod,value,geometry,date
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<POLYGON [°]>,<chr>
1500107,Abaetetuba,PA,1610.654,NA,32.27164,POLYGON ((-48.83139 -1.5635...,2020-12-01
1500800,Ananindeua,PA,190.581,NA,31.91007,POLYGON ((-48.33466 -1.2398...,2020-12-01
1501105,Bagre,PA,4397.321,NA,32.65541,POLYGON ((-50.29255 -1.8063...,2020-12-01
1501204,Baião,PA,3759.834,NA,32.88814,POLYGON ((-49.48643 -2.6631...,2020-12-01
1501303,Barcarena,PA,1310.338,NA,31.93690,POLYGON ((-48.4695 -1.59545...,2020-12-01
1501402,Belém,PA,1059.466,NA,31.35751,POLYGON ((-48.35304 -1.2210...,2020-12-01
1501501,Benevides,PA,187.826,NA,32.35474,POLYGON ((-48.33104 -1.2569...,2020-12-01
1501782,Breu Branco,PA,3941.904,1,32.63459,POLYGON ((-48.92561 -3.4129...,2020-12-01
1502103,Cametá,PA,3081.367,NA,32.47707,POLYGON ((-49.40003 -1.9237...,2020-12-01


### Rbind lists of dataframes in one big dataframe

##### If Health Data:

In [7]:
DATA_TYPE <- "dengue"
# DATA_TYPE <- "zika"


final_df <- do.call(rbind, dfs) %>%
  dplyr::select(cod, date, value) %>%
  rename(cd_mun = cod, !!DATA_TYPE := value)

dim(final_df)
head(final_df)

[1] 252   3

,cd_mun,date,dengue
,<chr>,<chr>,<chr>
1,150010,2020-12-01,0.0
2,150080,2020-12-01,0.0
3,150110,2020-12-01,0.0
4,150120,2020-12-01,0.0
5,150130,2020-12-01,0.79
6,150140,2020-12-01,1.47


##### If Climate Data:

In [14]:
DATA_TYPE <- "precipitation"
# DATA_TYPE <- "temperature"


final_df <- do.call(rbind, dfs) %>%
  dplyr::select(CD_MUN, date, value) %>%
  rename(cd_mun = CD_MUN, !!DATA_TYPE := value)

dim(final_df)
head(final_df)

[1] 252   3

,cd_mun,date,precipitation
,<chr>,<chr>,<dbl>
1,1500107,2020-12-01,32.27164
2,1500800,2020-12-01,31.91007
3,1501105,2020-12-01,32.65541
4,1501204,2020-12-01,32.88814
5,1501303,2020-12-01,31.93690
6,1501402,2020-12-01,31.35751


### Save data

In [9]:
FINAL_CSV_NAME <- "data_denguecases_months_2020.csv"

write.csv(final_df, FINAL_CSV_NAME, row.names = FALSE)